In [17]:
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras import models

In [6]:
# starting point 
my_model= models.Sequential()

# Add first convolutional block
my_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', 
                    input_shape=(178,218,3)))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# second block
my_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# third block
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# fourth block
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))


In [7]:
# global average pooling
my_model.add(GlobalAveragePooling2D())
# fully connected layer
my_model.add(Dense(64, activation='relu'))
my_model.add(BatchNormalization())
# make predictions
my_model.add(Dense(2, activation='sigmoid'))

In [8]:
# Show a summary of the model. Check the number of trainable parameters
my_model.summary()

# use early stopping to optimally terminate training through callbacks
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# save best model automatically
mc= ModelCheckpoint(r'C:\Users\User\Desktop\4th_year_AMLS\zceecyt-AMLSassignment19_20-16053203\dataset\celeba', monitor='val_loss', 
                    mode='min', verbose=1, save_best_only=True)
cb_list=[es,mc]


# compile model 
my_model.compile(optimizer='adam', loss='binary_crossentropy', 
                 metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 178, 218, 16)      448       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 89, 109, 16)       0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                1088      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 130       
Total params: 1,922
Trainable params: 1,794
Non-trainable params: 128
__________________________________________________

In [10]:
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [11]:
# set up data generator
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# get batches of training images from the directory
train_generator = data_generator.flow_from_directory(
        r'C:\Users\User\Desktop\4th_year_AMLS\zceecyt-AMLSassignment19_20-16053203\dataset\celeba',
        target_size=(178, 218),
        batch_size=12,
        class_mode='categorical')

Found 5000 images belonging to 1 classes.


In [14]:
# get batches of validation images from the directory
validation_generator = data_generator.flow_from_directory(r'C:\Users\User\Desktop\4th_year_AMLS\zceecyt-AMLSassignment19_20-16053203\dataset\celeba',  
        target_size=(178, 218),
        batch_size=12,
        class_mode='categorical')

Found 5000 images belonging to 1 classes.


In [15]:
history = my_model.fit_generator(
        train_generator,
        epochs=30,
        steps_per_epoch=2667,
        validation_data=validation_generator,
        validation_steps=667, callbacks=cb_list)

Epoch 1/30


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [16]:
# plot training and validation accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylim([.5,1.1])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig("Custom_Keras_ODSC.png", dpi=300)

NameError: name 'history' is not defined

In [18]:
####### Testing ################################

# load a saved model
from keras.models import load_model
import os
os.chdir('yourdirectory')
saved_model = load_model('Custom_Keras_CNN.h5')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'yourdirectory'

In [19]:
# generate data for test set of images
test_generator = data_generator.flow_from_directory(
        'C:/Users/w10007346/Pictures/Celeb_sets/test',
        target_size=(178, 218),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

# obtain predicted activation values for the last dense layer
import numpy as np
test_generator.reset()
pred=saved_model.predict_generator(test_generator, verbose=1, steps=1000)
# determine the maximum activation value for each sample
predicted_class_indices=np.argmax(pred,axis=1)

# label each predicted value to correct gender
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

# format file names to simply male or female
filenames=test_generator.filenames
filenz=[0]
for i in range(0,len(filenames)):
    filenz.append(filenames[i].split('\\')[0])
filenz=filenz[1:]

# determine the test set accuracy
match=[]
for i in range(0,len(filenames)):
    match.append(filenz[i]==predictions[i])
match.count(True)/1000

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/w10007346/Pictures/Celeb_sets/test'